In [2]:
import findspark   
findspark.init()

In [3]:
spark_url = 'local'
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark Tutorial')\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [4]:
path = 'bangkok_traffy.csv'
df = spark.read.csv(path, header=True, inferSchema=True, sep=',', encoding='utf-8', multiLine=True)

In [5]:
df.show(10, truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 ticket_id     | 2021-9LHDM6                                                                                                                                                                                                 